In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
import re
import pandas as pd
import os
import sys
import numpy as np
from opendssdirect.utils import run_command
import opendssdirect as dss

In [ ]:
from helpers import *

In [ ]:
!pwd

In [ ]:
current_dir = os.path.abspath('./')
!pwd

In [ ]:
OpenDSS_Folder = '{}'.format('/opendss_ephasor')

In [ ]:
OpenDSS_Folder

In [ ]:
OpenDSS_Master = '{}/{}'.format(OpenDSS_Folder,'Master.dss')

In [ ]:
OpenDSS_Master

In [ ]:
dss_path= OpenDSS_Master
run_command('compile {}'.format(dss_path))

In [ ]:
masterfile = read_dss_masterfile(OpenDSS_Master)
# get the name of the slack bus from the master.dss file
# masterfile

In [ ]:
# slackbus = masterfile['bus1'].split('.')[0]
slackbus = masterfile['circuit'].split('.')[0]
print(slackbus)

In [ ]:
tempdf = dss_to_df_diff('Master.dss')

In [ ]:
dfs = {}
act_keys = ['load','line', 'transformer','switch','pvsystem','capacitor','faultindicator','fuses']

In [ ]:
for x in tempdf:
    dataframe_key = '-'.join(set(x.keys())  & set(act_keys))
    if dataframe_key not in dfs.keys():
        dfs[dataframe_key] = pd.DataFrame()
    dfs[dataframe_key] = dfs[dataframe_key].append(x,ignore_index=True)    

In [ ]:
dfs.keys()

In [ ]:
for i in dfs:
    print(dfs[i].keys())

In [ ]:
reuired_sheetnames = ['Bus',
 'Current Injector',
 'Switch',
 'Vsource 3-phase',
 'Multiphase Line',
 'Multiphase Load',
 'Multiphase Shunt',
'Multiphase Transformer']
# template.sheet_names

sheet_name_cols = {}
for x in reuired_sheetnames:
    headers = [0]
    if str(x) == 'Multiphase Transformer':
        headers=[0, 1]
    temp = pd.read_excel('/Users/dvaidhyn/Desktop/ACES-COSIM/fy19cosim/aces-simulations/nodes/nodes-sce/temp/Open_dss_model_08_22_2018/ConvertedFiles/Template_V1_6.xls',sheet_name=x,header=headers)
    sheet_name_cols[str(x)] = list(temp.columns.values)

In [ ]:
# Multiplahse Transformer
# Transformer_df = dss_to_df('{}/{}'.format(OpenDSS_Folder,'Transformers.dss'))
Transformer_df = pd.DataFrame(dfs['transformer'])

transformer_split_list = [
    {'bus' : 'from'},\
    {'bus+1' : 'to' }
]

Transformer_df_final = split_phases_to_cols(transformer_split_list,Transformer_df)   
replacement_keys = ['from1','from2', 'from3','to1','to2','to3']
phase_replace(phases_123_abc,Transformer_df_final,replacement_keys)

print(Transformer_df_final.columns)
print(sheet_name_cols['Multiphase Transformer'])

Transformer_df_final.columns = [('Unnamed: 0_level_0', '%loadloss'),('Unnamed: 0_level_0', '%noloadloss'),('Unnamed: 0_level_0','bus'), ('Unnamed: 0_level_0','bus+1'), ('winding From','Conn. type'), ('winding To','Conn. type'), 
       ('winding From', 'Bus1'),('winding From', 'Bus2'),('winding From', 'Bus3'),
        ('winding From', 'V (kV)'), ('winding To', 'V (kV)'), ('winding From', 'S_base (kVA)'),('winding To', 'S_base (kVA)'),
        'maxtap', 'maxtap+1',
       'mintap', 'mintap+1', 'numtap', 'numtap+1', ('Unnamed: 0_level_0','num. phases'), 'tap','tap+1',
       ('winding To', 'Bus1'),('winding To', 'Bus2'),('winding To', 'Bus3'),('Unnamed: 0_level_0','ID'), 'wdg', 'wdg+1', 'windings', 'xhl']

Transformer_df_final.sort_values(('Unnamed: 0_level_0','num. phases'),axis=0, ascending=True, inplace=True)
Transformer_df_final = Transformer_df_final[  
[ ('winding From','Conn. type'), ('winding To','Conn. type'), 
       ('winding From', 'Bus1'),('winding From', 'Bus2'),('winding From', 'Bus3'),
        ('winding From', 'V (kV)'), ('winding To', 'V (kV)'), ('winding From', 'S_base (kVA)'),('winding To', 'S_base (kVA)'),
         ('Unnamed: 0_level_0','num. phases'),
       ('winding To', 'Bus1'),('winding To', 'Bus2'),('winding To', 'Bus3'),('Unnamed: 0_level_0','ID')]
]

Transformer_df_final [('Unnamed: 0_level_0','Tap1')] = 0
Transformer_df_final [('Unnamed: 0_level_0','Tap2')] = np.nan
Transformer_df_final [('Unnamed: 0_level_0','Tap3')] = np.nan
Transformer_df_final [('Unnamed: 0_level_0','X (pu)')] = 0.01
Transformer_df_final [('winding From','X (pu)')] = 0.01
Transformer_df_final [('winding To','X (pu)')] = 0.01
Transformer_df_final [('Unnamed: 0_level_0','Lowest Tap')] = -16
Transformer_df_final [('Unnamed: 0_level_0','Highest Tap')] = 16
Transformer_df_final [('Unnamed: 0_level_0','Min Range (%)')] = 10
Transformer_df_final [('Unnamed: 0_level_0','Max Range (%)')] = 10

phases3 = Transformer_df_final[('Unnamed: 0_level_0', 'num. phases')].loc[Transformer_df_final[('Unnamed: 0_level_0', 'num. phases')].str.contains('3')].index
Transformer_df_final[('Unnamed: 0_level_0','Tap2')][phases3]  = 0
Transformer_df_final[('Unnamed: 0_level_0','Tap3')][phases3] = 0

phases1 = Transformer_df_final[('Unnamed: 0_level_0', 'num. phases')].loc[Transformer_df_final[('Unnamed: 0_level_0', 'num. phases')].str.contains('1')].index
Transformer_df_final[('winding From', 'V (kV)')][phases1]  = Transformer_df_final[('winding From', 'V (kV)')][phases1].astype(float)*1.732
Transformer_df_final[('winding To', 'V (kV)')][phases1] = Transformer_df_final[('winding To', 'V (kV)')][phases1].astype(float)*1.732
Transformer_df_final.head()


In [ ]:
# Multiphase Loads
# Loads_df = dss_to_df('{}/{}'.format(OpenDSS_Folder,'Loads.dss'))
Loads_df = pd.DataFrame(dfs['load'])
loads_split_list = [
    {'bus1' : 'bus_'}
]
Loads_df_final = split_phases_to_cols(loads_split_list,Loads_df)
replacement_keys = ['bus_1','bus_2','bus_3']
phase_replace(phases_123_abc,Loads_df_final,replacement_keys)
temp_load = pd.DataFrame()
ld = dss.Loads.First()
while ld:
    z = {'load' : dss.Loads.Name(), 'kvar': dss.Loads.kvar(),'Bandwitdth(p.u)':dss.Loads.Vminpu() }
    temp_load = temp_load.append(z,ignore_index=True)
    ld = dss.Loads.Next()
Loads_df_final = pd.merge(Loads_df_final, temp_load, how='inner', on='load')  
temp = pd.DataFrame()
for i,x in Loads_df_final.iterrows():
    z = x
    for j in range(1,int(z['phases'])+1):
        z['P{}'.format(j)] = float(float(z['kw'])/float(z['phases']))
        z['Q{}'.format(j)] = float(float(z['kvar'])/float(z['phases']))
        z['conn.type'] = 'wye'
        z['type'] ='ZIP'
        z['k_z'] = 0
        z['k_i'] = 0
        z['k_p'] = 1
        z['status'] =1 
    temp = temp.append(z)
ephaseor_load_df = temp
# ephaseor_load_df.head()


print(ephaseor_load_df.columns)
print(sheet_name_cols['Multiphase Load'])
ephaseor_load_df.columns = ['Bandwitdth(p.u)', 'P1 (kW)', 'P2 (kW)', 'P3 (kW)', 'Q1 (kVAr)', 'Q2 (kVAr)', 'Q3 (kVAr)', 'bus1', 'Bus1',
       'Bus2', 'Bus3', 'Conn.type', 'K_i', 'K_p', 'K_z', 'V(kv)', 'kvar', 'kw',
       'ID', 'phases', 'Status', 'Type', 'yearly']

ephaseor_load_df.sort_values('phases',axis=0, ascending=True, inplace=True)
ephaseor_load_df = ephaseor_load_df[['Bandwitdth(p.u)', 'P1 (kW)', 'P2 (kW)', 'P3 (kW)', 'Q1 (kVAr)', 'Q2 (kVAr)', 'Q3 (kVAr)', 'bus1', 'Bus1',
       'Bus2', 'Bus3', 'Conn.type', 'K_i', 'K_p', 'K_z', 'V(kv)','ID', 'Status', 'Type' ]]
ephaseor_load_df ['Use initial voltage?'] = 0
ephaseor_load_df.head()

In [ ]:
# Switch
def switch_mod(df):
    df['ON_OFF'].replace(['true'],1,inplace=True)
    df['ON_OFF'].replace(['false'],0,inplace=True)
    df['From_bus'].replace('\.1','_a',regex=True,inplace=True)
    df['From_bus'].replace('\.2','_b',regex=True,inplace=True)
    df['From_bus'].replace('\.3','_c',regex=True,inplace=True)
    df['To_bus'].replace('\.1','_a',regex=True,inplace=True)
    df['To_bus'].replace('\.2','_b',regex=True,inplace=True)
    df['To_bus'].replace('\.3','_c',regex=True,inplace=True)
    df['Switch_ID'].replace('\.1','_a',regex=True,inplace=True)
    df['Switch_ID'].replace('\.2','_b',regex=True,inplace=True)
    df['Switch_ID'].replace('\.3','_c',regex=True,inplace=True)
    df['Switch_ID'].replace('line\.','line',regex=True,inplace=True)

Switch_key_map =  {'bus1' : 'From_bus', 'bus2' : 'To_bus', 'line' : 'Switch_ID', 'enabled' : 'ON_OFF', 'phases' : 'phases'}


try:
    Switch_ephasror_final = pd.DataFrame(dfs['switch-line'] )
except:    
    Switch_ephasror_final = pd.DataFrame(dfs['line-switch'] )

Switch_ephasror_final = Switch_ephasror_final.append(dfs['line'].loc[dfs['line'].length == '1.0'],ignore_index=True)

col_replace(Switch_ephasror_final, Switch_key_map)

add_on_df = pd.DataFrame()
add_on = []
for i,x in Switch_ephasror_final.iterrows():
    names = x['From_bus'].split('.')
    switch_name = x['Switch_ID'].split('.')[0]
    if len(names) > 2:
        z = x
        tonames = z['To_bus'].split('.')   
#         print(names[1:])
        for y in names[1:]:
            temprow = {}
            temprow = z
            temprow['From_bus'] = '{}.{}'.format(names[0],y)
            temprow['To_bus'] = '{}.{}'.format(tonames[0],y)    
            temprow['Switch_ID'] = '{}.{}'.format(switch_name,y)
#             print(temprow['Switch_ID'])
            add_on_df = add_on_df.append(temprow,ignore_index=True)


Switch_ephasror_final.drop(Switch_ephasror_final.loc[Switch_ephasror_final.phases.map(lambda x: int(x)) > 1 ].index , inplace=True)
Switch_ephasror_final = Switch_ephasror_final.append(add_on_df,ignore_index=True)
switch_mod(Switch_ephasror_final)

print(Switch_ephasror_final.columns)
print(sheet_name_cols['Switch'])

Switch_ephasror_final.columns  = ['From Bus','Status', 'ID','To Bus', 'phases']
Switch_ephasror_final.head()

In [ ]:
# Mulitphase Shunt
def capacitor_mod(df):
    extra_cols = ['Bus1', 'Bus2', 'Bus3','P1','P2','P3','Q1','Q2','Q3','V','Status1','Status2','Status3']
    ndf = pd.DataFrame(columns=extra_cols)
    ndf = ndf.append(df,ignore_index=True,sort=False)
    for i in range(ndf.shape[0]):
            ndf.loc[i,'Status1'] = 1
            ndf.loc[i,'Status2'] = 1
            ndf.loc[i,'Status3'] = 1
            ndf.loc[i]['P1'] = 0
            ndf.loc[i]['P2'] = 0
            ndf.loc[i]['P3'] = 0
            ndf.loc[i]['V'] = ndf.loc[i]['kv']
            ndf.loc[i]['Q1'] = ndf.loc[i]['kvar']
            ndf.loc[i]['Q2'] = ndf.loc[i]['kvar']
            ndf.loc[i]['Q3'] = ndf.loc[i]['kvar'] 
            if int(ndf.loc[i]['phases'] )== 3:
                ndf.loc[i]['Bus1'] = str(ndf.loc[i]['bus1']).split(".")[0]+str('_a')
                ndf.loc[i]['Bus2'] = str(ndf.loc[i]['bus1']).split(".")[0]+str('_b')
                ndf.loc[i]['Bus3'] = str(ndf.loc[i]['bus1']).split(".")[0]+str('_c')
                ndf.loc[i]['Q1'] = float(ndf.loc[i]['kvar'])*0.3333
                ndf.loc[i]['Q2'] = float(ndf.loc[i]['kvar'])*0.3333
                ndf.loc[i]['Q3'] = float(ndf.loc[i]['kvar'])*0.3333  
    ndf.drop(['bus1','kv','kvar'],inplace=True, axis=1)   
    return ndf
    

# Capacitors_df = dss_to_df('{}/{}'.format(OpenDSS_Folder,'Capacitors.dss'))
Capacitors_df = dfs['capacitor']
Capacitors_df = capacitor_mod(Capacitors_df)

print(Capacitors_df.columns)
print(sheet_name_cols['Multiphase Shunt'])

Capacitors_df.sort_values('phases',axis=0, ascending=True, inplace=True)

Capacitors_df.columns = ['Bus1' ,'Bus2', 'Bus3', 'P1(kW)', 'P2(kW)' ,'P3(kW)', 'Q1(kVAr)' ,'Q2(kVAr)' ,'Q3(kVAr)' ,'V(kV)' ,'Status1'
 ,'Status2' ,'Status3', 'ID', 'conn' ,'phases']
Capacitors_df = Capacitors_df[['Bus1' ,'Bus2', 'Bus3', 'P1(kW)', 'P2(kW)' ,'P3(kW)', 'Q1(kVAr)' ,'Q2(kVAr)' ,'Q3(kVAr)' ,'V(kV)' ,'Status1'
 ,'Status2' ,'Status3', 'ID']]

Capacitors_df.head()

In [ ]:
# Current Injector

# PVsystems_df = dss_to_df('{}/{}'.format(OpenDSS_Folder,'PVsystems.dss'))
PVsystems_df = pd.DataFrame(dfs['pvsystem'])
PVsystems_key_map = {'bus1' :'Bus', 'kv' : 'kV_rating', 'kva' : 'kVA_rating' , \
                     'pf' : 'Power_factor', 'phases': 'Phases', 'pvsystem':'PV_ID' }
col_replace(PVsystems_df, PVsystems_key_map) 

pv_addon =pd.DataFrame()
for i,x in PVsystems_df.loc[PVsystems_df.Phases.astype(float) > 1].iterrows():
    y = {}
    y = x
    names = x['Bus']
    base_name = names.split('.')[0]
    base_pv_name = x['PV_ID'].split('.')[0]
    for k in names.split('.')[1:]:
        y['Bus'] = '{}.{}'.format(base_name,k)
        y['PV_ID'] = '{}.{}'.format(base_pv_name,k)    
        pv_addon = pv_addon.append(y,ignore_index=True)

PVsystems_df.drop(PVsystems_df.loc[PVsystems_df.Phases == '3'].index, inplace=True)
PVsystems_df = PVsystems_df.append(pv_addon,ignore_index=True)
replacement_keys = ['Bus','PV_ID']
phase_replace(phases_123_abc,PVsystems_df,replacement_keys)

print(PVsystems_df.columns)
print(sheet_name_cols['Current Injector'])

PVsystems_df.sort_values('Phases',axis=0, ascending=True, inplace=True)
PVsystems_df.columns = ['Bus','ID', 'Phases','Power_factor', 'kVA_rating','kV_rating']
PVsystems_df = PVsystems_df [['Bus','ID']]

PVsystems_df.head()

In [ ]:
# Mulitphase Lines
# Lines_df = dss_to_df('{}/{}'.format(OpenDSS_Folder,'Lines.dss'))
Lines_df = dfs['line']

lines_split_list = [
    {'bus1' : 'from'},\
    {'bus2' : 'to' }
]
Lines_df_final = split_phases_to_cols(lines_split_list,Lines_df)  
replacement_keys = ['from1','from2', 'from3','to1','to2','to3']
phase_replace(phases_123_abc,Lines_df_final, replacement_keys)
temprxc = []
tempb0b1 = []
run_command('solve')    
Line = dss.Lines.First()
while Line:
    x = {}
    
    Name = dss.Lines.Name()
    x['line'] = Name

    x['b0'] =  dss.Lines.C0()
    x['b1'] =  dss.Lines.C1()
    
    R = dss.Lines.RMatrix()
    Ydim = int(np.sqrt(len(R)))
    for i in range(1,Ydim+1): 
        for j in range(1,Ydim+1): 
            x['r{}{}'.format(i,j)]  =  R[(i-1)*Ydim + (j-1)]
    
    X = dss.Lines.XMatrix()
    Ydim = int(np.sqrt(len(X)))
    for i in range(1,Ydim+1): 
        for j in range(1,Ydim+1): 
            x['x{}{}'.format(i,j)] = X[(i-1)*Ydim + (j-1)]
    
    C = dss.Lines.CMatrix()
    Ydim = int(np.sqrt(len(C)))
    for i in range(1,Ydim+1): 
        for j in range(1,Ydim+1): 
            x['c{}{}'.format(i,j)] =  C[(i-1)*Ydim + (j-1)]
      
    temprxc.append(x)
    Line = dss.Lines.Next()
    
rxc = pd.DataFrame(temprxc)  

#  drop all lines of length 1 from the lines dataframe
Lines_df_final.drop(Lines_df_final.loc[Lines_df_final.length == '1.0'].index, inplace=True)

Lines_df_final = pd.merge(Lines_df_final, rxc, how='inner', on='line')

print(Lines_df_final.columns)
print(sheet_name_cols['Multiphase Line'])

Lines_df_final.columns = ['b1(uS/length_unit)', 'bus1', 'bus2', 'enabled', 'From1', 'Length (length_unit)', 'ID',
       'normamps', 'phases', 'r0(ohm/length_unit)', 'r1(ohm/length_unit)', 'To1', 'units', 'x0(ohm/length_unit)', 'x1(ohm/length_unit)', 'From2',
       'From3', 'To2', 'To3', 'b0(uS/length_unit)', 'b1_y', 'c11(uS/length_unit)', 'c12(uS/length_unit)', 'c13(uS/length_unit)', 'c21(uS/length_unit)', 'c22(uS/length_unit)',
       'c23(uS/length_unit)', 'c31(uS/length_unit)', 'c32(uS/length_unit)', 'c33(uS/length_unit)', 'r11(ohm/length_unit)', 'r12(ohm/length_unit)', 'r13(ohm/length_unit)', 'r21(ohm/length_unit)', 'r22(ohm/length_unit)', 'r23(ohm/length_unit)',
       'r31(ohm/length_unit)', 'r32(ohm/length_unit)', 'r33(ohm/length_unit)', 'x11(ohm/length_unit)', 'x12(ohm/length_unit)', 'x13(ohm/length_unit)', 'x21(ohm/length_unit)', 'x22(ohm/length_unit)', 'x23(ohm/length_unit)', 'x31(ohm/length_unit)',
       'x32(ohm/length_unit)', 'x33(ohm/length_unit)']

Lines_df_final.sort_values('phases',axis=0, ascending=True, inplace=True)
Lines_df_final = Lines_df_final[['b1(uS/length_unit)', 'From1', 'Length (length_unit)', 'ID',
      'r0(ohm/length_unit)', 'r1(ohm/length_unit)', 'To1','x0(ohm/length_unit)', 'x1(ohm/length_unit)', 'From2',
       'From3', 'To2', 'To3', 'b0(uS/length_unit)', 'c11(uS/length_unit)',  'c21(uS/length_unit)', 'c22(uS/length_unit)',
       'c31(uS/length_unit)', 'c32(uS/length_unit)', 'c33(uS/length_unit)', 'r11(ohm/length_unit)',  'r21(ohm/length_unit)', 'r22(ohm/length_unit)', 
       'r31(ohm/length_unit)', 'r32(ohm/length_unit)', 'r33(ohm/length_unit)', 'x11(ohm/length_unit)', 'x21(ohm/length_unit)', 'x22(ohm/length_unit)', 'x31(ohm/length_unit)',
       'x32(ohm/length_unit)', 'x33(ohm/length_unit)']]
Lines_df_final['mode'] = 'FULL'
Lines_df_final

Lines_df_final.head()

In [ ]:
# # VSource
# vs = dss.Vsources.First()
# while vs:
#     print(dss.Vsources.Name(),dss.Vsources.BasekV(),dss.Vsources.PU(),dss.Vsources.AngleDeg(),dss.Vsources.Frequency(),dss.Vsources.Phases())
#     vs = dss.Vsources.Next()


In [ ]:
# Bus
phases = ['a','b','c']
Bus_ephasor_df = pd.DataFrame()
ll = dss.Circuit.AllBusNames()
for x in ll:
    actbus = dss.Circuit.SetActiveBus(x)
    z = {}
    z['BaseKV'] = dss.Bus.kVBase()    
    mag_angle = dss.Bus.VMagAngle()
    vpu = dss.Bus.puVmagAngle()
    nodes = dss.Bus.Nodes()
    for ti in range(0,len(nodes),1):     
        z['Node'] = nodes[ti]
        z['Bus']  = '{}_{}'.format(dss.Bus.Name(),(phases[nodes[ti]-1]))
        t = ti*2
        z['VoltageMagnitude'] = mag_angle[t]
        z['PhaseAngle'] = mag_angle[t+1] 
        z['VoltagePU'] = vpu[t]
        Bus_ephasor_df = Bus_ephasor_df.append(z,ignore_index=True)

        
print(Bus_ephasor_df.columns)
print(sheet_name_cols['Bus'])

Bus_ephasor_df.columns= ['Base Voltage (V)','Bus','Node','Angle (deg)','Voltage (V)','Voltage (V, pu)']
Bus_cols = ['Base Voltage (V)','Bus','Angle (deg)','Voltage (V, pu)','Voltage (V)']
Bus_ephasor_df = Bus_ephasor_df[Bus_cols]
Bus_ephasor_df['Type'] = 'PQ'
Bus_ephasor_df['Type'][Bus_ephasor_df.loc[Bus_ephasor_df['Bus'].str.contains('{}*'.format(slackbus),regex=True,case=False) ].index] = 'SLACK'
Bus_ephasor_df.loc[Bus_ephasor_df['Bus'].str.contains('{}*'.format(slackbus),regex=True,case=False) ].index
Bus_ephasor_df.head()

Bus_ephasor_df['Base Voltage (V)'] = Bus_ephasor_df['Base Voltage (V)']*1000

Bus_ephasor_df
        
        
Bus_ephasor_df.head()

In [ ]:
reuired_sheetnames = ['Bus',
 'Current Injector',
 'Switch',
 'Vsource 3-phase',
 'Multiphase Line',
 'Multiphase Load',
 'Multiphase Shunt',
'Multiphase Transformer']
# template.sheet_names

sheet_name_cols = {}
for x in reuired_sheetnames:
    headers = [0]
    if str(x) == 'Multiphase Transformer':
        headers=[0, 1]
    temp = pd.read_excel('Template_V1_6.xls',sheet_name=x,header=headers)
    sheet_name_cols[str(x)] = list(temp.columns.values)

In [ ]:
['Bus',
 'Current Injector',
 'Switch',
 'Vsource 3-phase',
 'Multiphase Line',
 'Multiphase Load',
 'Multiphase Shunt',
'Multiphase Transformer']

In [ ]:
dataframe_to_sheets = {}
dataframe_to_sheets['Bus'] = Bus_ephasor_df
dataframe_to_sheets['Current Injector'] = PVsystems_df
dataframe_to_sheets['Switch'] = Switch_ephasror_final
dataframe_to_sheets['Multiphase Shunt'] = Capacitors_df
dataframe_to_sheets['Multiphase Load'] = ephaseor_load_df
dataframe_to_sheets['Multiphase Line'] = Lines_df_final
dataframe_to_sheets['Multiphase Transformer'] = Transformer_df_final

In [ ]:
EPHASOR_OUTPUT_FILE = 'ephasorfile.xlsx'

In [ ]:
# Write to EPhasor model to excel sheets
writer = pd.ExcelWriter(EPHASOR_OUTPUT_FILE, engine='xlsxwriter')
dataframe_to_sheets['Multiphase Line'].to_excel(writer,sheet_name="MultiplhaseLines",index=False)
dataframe_to_sheets['Multiphase Load'].to_excel(writer,sheet_name="MultiphaseLoads",index=False)
dataframe_to_sheets['Switch'].to_excel(writer,sheet_name="Switch",index=False)
dataframe_to_sheets['Multiphase Shunt'].to_excel(writer,sheet_name="MultiplaseShunt",index=False)
dataframe_to_sheets['Current Injector'].to_excel(writer,sheet_name="CurrentInjector",index=False)
dataframe_to_sheets['Multiphase Transformer'].to_excel(writer,sheet_name="MultiphaseTransformer",index=False)
dataframe_to_sheets['Bus'].to_excel(writer,sheet_name="Bus",index=False)
writer.save()

In [ ]:
!open ./ephasorfile.xlsx